In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('/kaggle/input/cleaned-from-raw/train_cleaned.csv')
test = pd.read_csv('/kaggle/input/cleaned-from-raw/test_cleaned.csv')

In [3]:
label = pd.read_csv('/kaggle/input/cleaned-from-raw/mask_to_actual_labeling.csv')
label = label[label['masked_column'].isin(train.columns)]

In [14]:
for col in train.columns:
    if col not in label['masked_column'].to_list():
        print(col)

day
dayofweek
hour
minute
second
is_month_start
is_month_end
is_weekend


In [9]:
train.drop(columns=['year','month','quarter','weekofyear'],inplace=True)
test.drop(columns=['year','month','quarter','weekofyear'],inplace=True)

In [15]:
cols = ['day','dayofweek','hour','minute','second','is_month_start','is_month_end','is_weekend']
desc = ['day','dayofweek','hour','minute','second','is_month_start','is_month_end','is_weekend']
types = ['Categorical','Categorical','Categorical','Categorical','Categorical','Categorical','Categorical','Categorical',]
to_add = pd.DataFrame({'masked_column':cols,'Description':desc,'Type':types})
label = pd.concat([label,to_add],axis=0)

In [17]:
train.shape

(770164, 317)

In [19]:
from sklearn.feature_selection import mutual_info_classif
num = label[label['Type']=='Numerical']['masked_column'].to_list()
num.remove('id5')
X = train[num]
y = train['y']
scores = []
dic = {}
for i in range(0,X.shape[1]+10,10):
    mi_scores = mutual_info_classif(X[X.columns[i:i+10]], y, discrete_features=False, random_state=0)
    scores.extend(mi_scores)
    for j in range(0,10,1):
        try:
            print(str(X.columns[i+j])+" , "+str(mi_scores[j]))
            dic[X.columns[i+j]] = mi_scores[j]
        except:
            print("out of range")

f1 , 0.0030747559081327758
f2 , 0.003192813781031978
f3 , 0.0012544370700682261
f4 , 0.0011272994270388192
f5 , 0.004037919444914495
f6 , 0.004918224884668199
f7 , 0.0023566972807427256
f8 , 0.003495919767418254
f9 , 0.0039028841370472023
f10 , 0.004207103659204847
f11 , 0.00379309665338301
f12 , 0.003620840161389749
f13 , 0.16668095774258085
f18 , 0.00018765236732254387
f22 , 0.005675020185807567
f26 , 0.001888782102016151
f27 , 0.002644824309941063
f28 , 0.0950934746568326
f29 , 0.10973497277368494
f30 , 0.09189081356686568
f31 , 0.08933841841560297
f32 , 0.03193010078215042
f33 , 0.00020846419963116425
f34 , 0.00016413719404595462
f35 , 0.08378541445806542
f36 , 0.0023241448749515836
f37 , 0.0
f38 , 0.008702536908409275
f39 , 0.06393679083821335
f40 , 0.0207916182591823
f41 , 0.07662445206952273
f43 , 0.024733539861916842
f44 , 0.0019128387370610511
f45 , 0.0029754671663575483
f46 , 0.050873058866932164
f47 , 0.05451523587660212
f49 , 0.014077609017513648
f51 , 0.05017875785417392
f

ValueError: at least one array or dtype is required

In [22]:
dropped = []
for col,score in dic.items():
    if score<=0.005:
        dropped.append(col)
train.drop(columns=dropped,inplace=True)
test.drop(columns=dropped,inplace=True)
label = label[label['masked_column'].isin(train.columns)]

In [26]:
num = label[label['Type']=='Numerical']['masked_column'].to_list()
num.remove('id5')
corr_matrix = train[num].corr()

In [33]:
cols = num.copy()
def find_col(col):
    for pos in range(len(lis)):
        if col in lis[pos]:
            return pos

check = {}
for col in cols:
    check[col]=0
lis=[]
cnt=0
for i in range(len(cols)):
    for j in range(len(cols)):
        if i!=j:
            if abs(corr_matrix[cols[i]][cols[j]])>=0.85:
                cnt+=1
                if check[cols[i]]==0 and check[cols[j]]==0:
                    lis.append([cols[i],cols[j]])
                    check[cols[i]]=1
                    check[cols[j]]=1
                elif check[cols[i]]==0:
                    pos = find_col(cols[j])
                    lis[pos].append(cols[i])
                    check[cols[i]]=1
                    check[cols[j]]=1
                elif check[cols[j]]==0:
                    pos = find_col(cols[i])
                    lis[pos].append(cols[j])
                    check[cols[i]]=1
                    check[cols[j]]=1
                else:
                    continue
                    # it will be handled in the next correlation matrix
print(cnt)

234


In [35]:
for l in lis:
    for col in l:
        print(label[label['masked_column']==col]['Description'].to_list()[0])
    print()

Cumulative impressions (with decaying weights for impressions in the past) of a cust_xref_id for a merchant offer in the last 30 days 
No. of impressions for a CM in the last 14 days where past impressions get decaying weights
No of impressions for a CM in the last 30 days where impressions days get decaying weights

Spend in Restaurant in the last 3 months
context variable fetched in real time where this is Number of seconds after 00:00 hrs 

Total time spent on all pages in the last 30 days
Total time spent on pages NOT IN ANY OF THE ABOVE ONES in the last 30 days

Total time spent on all pages in the last 180 days
Total time spent in pages NOT IN ANY OF THE ABOVE ONES in the last 180 days

If offer type benefit awareness then impressions  in last 180 days
If offer type Acquisition-Others then impressions in last 180 days
If offer type Lending/TLI then impressions in last 180 days
Total impressions in all non-merchant categories for a CM in the last 180 days
No of clicks for a CM in 

In [36]:
from sklearn.preprocessing import LabelEncoder
dropped=[]
for l in lis:
    cols=l.copy()
    if label[label['masked_column']==cols[0]]['Type'].to_list()[0]=='Numerical':
        X = train[cols].copy()
        y = train['y']
        mi_scores = mutual_info_classif(X,y,discrete_features=False,random_state=42)
        maxi=0
        copy = cols.copy()
        for col,score in zip(copy,mi_scores):
            if score>maxi:
                maxi=score
                keep=col
        cols.remove(keep)
        dropped.extend(cols)
    elif label[label['masked_column']==cols[0]]['Type'].to_list()[0]=='Categorical':
        X = train[cols].copy()
        y = train['y']
        for col in X.columns:
            X[col] = LabelEncoder().fit_transform(X[col])
        mi_scores = mutual_info_classif(X,y,discrete_features=True,random_state=42)
        maxi=0
        copy = cols.copy()
        for col,score in zip(copy,mi_scores):
            if score>maxi:
                maxi=score
                keep=col
        cols.remove(keep)
        dropped.extend(cols)

In [2]:
train.drop(columns=dropped,inplace=True)
test.drop(columns=dropped,inplace=True)

NameError: name 'train' is not defined

In [37]:
train.to_csv('train_cut_down.csv',index=False)
test.to_csv('test_cut_down.csv',index=False)

In [38]:
label.to_csv('updated_label.csv',index=False)